In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import SPTCata as spt
import matplotlib.pyplot as plt
import numpy as np
import os, sys
import re
from collections import defaultdict

# for fitting
from scipy.optimize import curve_fit

# 2019-05-09 Diffusion analysis
I proceed with the analysis of the diffusion coefficients.

## Load data

In [ ]:
# directories
datadirs = {
    'Olaparib-R5020' : '../data/1_Olaparib_R5020',
    'R5020' : '../data/2_DMSO_R5020_Control',
    'EtOH' : '../data/3_EtOH_Nohormone_Control'
}

# scale
scale_l = 0.160
dt = 0.015

# cycle through all the directories and do the analysis
experiments = defaultdict(list)
data = defaultdict(list)
for treatment, datadir in datadirs.items() :
    for subdir in os.listdir(datadir) :
        full_dir_name = '%s/%s'%(datadir, subdir)
        # print("Processing %s"%(full_dir_name))
        experiment = spt.SPT(full_dir_name, scale_l, quality=50, links=False)
        experiments[treatment].append(experiment)
        trajectories = experiment.trajectory_spots
        data[treatment].extend(trajectories)

## Diffusion coefficient

To calculate the diffusion coefficient, I'll use the fitting of the distribution of displacements.

First, I gather the information on the square displacements in different treatments.

In [ ]:
# calculate the distribution of r2 for all the tracks
displacements2 = defaultdict(list)
for treatment, tracks in experiments.items() :
    for track in tracks :
        displacements2[treatment].extend(np.array(track.displacement_spots)**2)

I define then two competitor models for the diffusion: a simple one-species diffusion model, and a two-species diffusion model.

In [ ]:
def diffusion_model_1s(X, *p):
    """
    One species diffusion model.
    """
    D = p[0]
    x = X[0]
    dt = X[1]
    return 1-np.exp(-x/(4*D*dt))


def diffusion_model_2s(X, *p) :
    """
    Two species diffusion model.
    """
    D1, D2, f1 = p
    x = X[0]
    dt = X[1]
    return 1-f1*np.exp(-x/(4*D1*dt))-(1-f1)*np.exp(-x/(4*D2*dt))

Now we're ready to perform the fits.

In [ ]:
def fit_diffusion_models(r2_tracks, m, M, N, dt,
                         p0_1s = 10.0, p0_2s = [10.0, 1.0, 0.1]) :
    
    # calculate the histogram of square displacements of the tracks
    bins = np.linspace(m, M, N+1)
    h, bins = np.histogram(r2_tracks, bins=bins)
    
    # prepare x and y for fitting
    delta = bins[1]-bins[0]
    x = np.zeros(N+1)
    y = np.zeros(N+1)
    x = bins + delta/2.
    y[1:] = np.cumsum(h)/sum(h)
    X = (x, dt)
    
    # one species diffusion model fit
    coeff_1s, var_matrix_1s = curve_fit(diffusion_model_1s, X, y, p0=p0_1s)
    
    # two species diffusion model fit
    coeff_2s, var_matrix_2s = curve_fit(diffusion_model_2s, X, y, p0=p0_2s,
                                       bounds = (0,
                                                 [np.inf, np.inf, 1.0]))
    
    return x, y, coeff_1s, var_matrix_1s, coeff_2s, var_matrix_2s

We're now ready to plot everything. Let's make nice plots.

In [ ]:
# for aesthetics
import matplotlib
matplotlib.rcParams['text.usetex'] = True

In [ ]:
N = 200

for treatment, displacement2 in displacements2.items() :
    
    x, y, coeff_1s, var_matrix_1s, coeff_2s, var_matrix_2s =\
    fit_diffusion_models(displacement2, 0., 0.05, N, dt)
    
    fig = plt.figure(figsize=(6,4))
    plt.plot(x, y, 'o', markersize=5, label='Data')
    plt.plot(x, diffusion_model_1s((x, dt), coeff_1s[0]), label='One species fit')
    plt.plot(x, diffusion_model_2s((x, dt), coeff_2s[0],
                                      coeff_2s[1],
                                      coeff_2s[2]), label='Two species fit')
    plt.title(treatment, fontsize=24)
    plt.axhline(y = 1.0, linestyle = '--', color = 'k', linewidth = 0.75)
    plt.xlabel("Displacement [$\mu$m]")
    plt.ylabel("Cumulative distribution")
    plt.xlim(0., 0.02)
    plt.xticks(np.arange(0., 0.0205, 0.005))
    plt.legend(loc='lower right', fontsize = 16)
    
    plt.text(0.007, 0.8, "One species fit: D = %.3f $\mu m^2/s$"%(coeff_1s[0]), fontsize = 14)
    plt.text(0.007, 0.5, "Two species fit: D1 = %.3f $\mu m^2/s$\n"
                          "                D2 = %.3f $\mu m^2/s$\n"
                          "                f1 = %.3f"%(coeff_2s[0],
                                                                coeff_2s[1],
                                                                coeff_2s[2]), fontsize = 14)
    
    plt.show()

## Angle Analysis


In [ ]:
angles = defaultdict(list)
for treatment, experiment_list in experiments.items() :
    for experiment in experiment_list :
        experiment.angle_analysis()
        angles[treatment].extend(experiment.angles)
    angles[treatment] = np.array(angles[treatment])
    angles[treatment] = angles[treatment][~np.isnan(angles[treatment])]

In [ ]:
for treatment in angles.keys() :
    fig = spt.plot_angles(angles[treatment], 24, treatment)
    plt.show()